In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns


In [11]:
#Sql query 
engine = create_engine("sqlite:///mta_data.db")
df = pd.read_sql("SELECT * FROM mta_data", engine)

In [12]:
# Checking Dataframe's first 5 rows:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,03:00:00,REGULAR,7672966,2645023
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,07:00:00,REGULAR,7672969,2645040
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,11:00:00,REGULAR,7672991,2645127
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,15:00:00,REGULAR,7673037,2645193
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,19:00:00,REGULAR,7673097,2645270


In [13]:
# Checking the dataframe size:
df.shape

(2732603, 11)

In [14]:
# Checking general info about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2732603 entries, 0 to 2732602
Data columns (total 11 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   C/A       object
 1   UNIT      object
 2   SCP       object
 3   STATION   object
 4   LINENAME  object
 5   DIVISION  object
 6   DATE      object
 7   TIME      object
 8   DESC      object
 9   ENTRIES   int64 
 10  EXITS     int64 
dtypes: int64(2), object(9)
memory usage: 229.3+ MB


In [15]:
#Checking basic descriptive statistics 
df.describe()

,ENTRIES,EXITS
count,2.732603e+06,2.732603e+06
mean,4.201794e+07,3.386391e+07
std,2.204692e+08,1.945824e+08
min,0.000000e+00,0.000000e+00
25%,2.330550e+05,1.144150e+05
50%,1.369239e+06,8.555220e+05
75%,6.007727e+06,3.992825e+06
max,2.147375e+09,2.122704e+09


## Data Cleaning:

In [17]:
df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [18]:
df.columns = [column.strip().lower().replace('/','_') for column in df.columns] #formatting column names
df.head()

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,03:00:00,REGULAR,7672966,2645023
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,07:00:00,REGULAR,7672969,2645040
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,11:00:00,REGULAR,7672991,2645127
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,15:00:00,REGULAR,7673037,2645193
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,19:00:00,REGULAR,7673097,2645270


In [19]:
# Turn the dataframe into a time series
df["date_time"] = pd.to_datetime(df.date + " " + df.time, format="%m/%d/%Y %H:%M:%S")
df.head()

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,03:00:00,REGULAR,7672966,2645023,2021-12-18 03:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,07:00:00,REGULAR,7672969,2645040,2021-12-18 07:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,11:00:00,REGULAR,7672991,2645127,2021-12-18 11:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,15:00:00,REGULAR,7673037,2645193,2021-12-18 15:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,19:00:00,REGULAR,7673097,2645270,2021-12-18 19:00:00


In [20]:
# Sanity Check to verify that "c_a", "unit", "scp", "station", "date_time" is unique
(df
 .groupby(["c_a", "unit", "scp", "station", "date_time"])
 .entries.count()
 .reset_index()
 .sort_values("entries", ascending=False)).head(5)

,c_a,unit,scp,station,date_time,entries
405945,G001,R151,00-05-01,CONEY IS-STILLW,2021-11-08 00:00:00,2
2710700,S101,R070,00-00-02,ST. GEORGE,2021-10-02 00:00:00,2
2710688,S101,R070,00-00-02,ST. GEORGE,2021-09-30 00:00:00,2
2710714,S101,R070,00-00-02,ST. GEORGE,2021-10-04 08:00:00,2
2710691,S101,R070,00-00-02,ST. GEORGE,2021-09-30 12:00:00,2


In [21]:
# Get rid of the duplicate entry
df.sort_values(["c_a", "unit", "scp", "station", "date_time"], inplace=True, ascending=False)
df.drop_duplicates(subset=["c_a", "unit", "scp", "station", "date_time"], inplace=True)


In [22]:
df.head()

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time
210382,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/24/2021,20:00:00,REGULAR,5562,994,2021-12-24 20:00:00
210381,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/24/2021,16:00:00,REGULAR,5562,994,2021-12-24 16:00:00
210380,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/24/2021,12:00:00,REGULAR,5562,979,2021-12-24 12:00:00
210379,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/24/2021,08:00:00,REGULAR,5562,979,2021-12-24 08:00:00
210378,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/24/2021,04:00:00,REGULAR,5562,979,2021-12-24 04:00:00


In [10]:
# Sanity Check to verify that "c_a", "unit", "scp", "station", "date_time" is unique
(df.groupby(["c_a", "unit", "scp", "station", "date_time"]).entries.count().reset_index().sort_values("entries", ascending=False)).head(5)



,c_a,unit,scp,station,date_time,entries
0,A002,R051,02-00-00,59 ST,2021-09-25 00:00:00,1
1821692,R141,R031,00-03-02,34 ST-PENN STA,2021-12-21 15:00:00,1
1821694,R141,R031,00-03-02,34 ST-PENN STA,2021-12-21 23:00:00,1
1821695,R141,R031,00-03-02,34 ST-PENN STA,2021-12-22 03:00:00,1
1821696,R141,R031,00-03-02,34 ST-PENN STA,2021-12-22 07:00:00,1


No more duplicate Entries

In [23]:
#Checking for null values based on column
df.isnull().sum()

c_a          0
unit         0
scp          0
station      0
linename     0
division     0
date         0
time         0
desc         0
entries      0
exits        0
date_time    0
dtype: int64

No null values

In [24]:
df.describe()

,entries,exits
count,2.732548e+06,2.732548e+06
mean,4.201876e+07,3.386458e+07
std,2.204713e+08,1.945843e+08
min,0.000000e+00,0.000000e+00
25%,2.330550e+05,1.144300e+05
50%,1.369321e+06,8.555615e+05
75%,6.007816e+06,3.993015e+06
max,2.147375e+09,2.122704e+09


In [26]:
#Different dfs for train stations:
df_train_stops = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/Stations.csv')
df_train_stops.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,NaN
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,NaN


In [28]:
len(df_train_stops['Stop Name'].unique())

379

In [29]:
len(df['station'].unique())

379

In [31]:

df_train_stops.Borough.unique()

array(['Q', 'M', 'Bk', 'Bx', 'SI'], dtype=object)

In [36]:
Bronx_df = df_train_stops[df_train_stops["Borough"] == "Bx"]
Bronx_df.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes
210,210,210,D01,IND,Concourse,Norwood-205 St,Bx,D,Subway,40.874811,-73.878855,NaN,Manhattan,0,NaN
211,211,211,D03,IND,Concourse,Bedford Park Blvd,Bx,B D,Subway,40.873244,-73.887138,Norwood - 205 St,Manhattan,1,NaN
212,212,212,D04,IND,Concourse,Kingsbridge Rd,Bx,B D,Subway,40.866978,-73.893509,Bedford Pk Blvd & 205 St,Manhattan,1,NaN
213,213,213,D05,IND,Concourse,Fordham Rd,Bx,B D,Subway,40.861296,-73.897749,Bedford Pk Blvd & 205 St,Manhattan,0,NaN
214,214,214,D06,IND,Concourse,182-183 Sts,Bx,B D,Subway,40.856093,-73.900741,Bedford Pk Blvd & 205 St,Manhattan,0,NaN


In [40]:
len(Bronx_df["Stop Name"].unique())

62

In [41]:
Brooklyn_df = df_train_stops[df_train_stops["Borough"] == "Bk"]
Brooklyn_df.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes
23,24,620,R28,BMT,Broadway,Court St,Bk,R,Subway,40.694100,-73.991777,Manhattan,Bay Ridge - 95 St,0,NaN
24,25,636,R29,BMT,Broadway,Jay St-MetroTech,Bk,R,Subway,40.692180,-73.985942,Manhattan,Bay Ridge - 95 St,1,NaN
25,26,26,R30,BMT,Broadway - Brighton,DeKalb Av,Bk,B Q R,Subway,40.690635,-73.981824,Manhattan,Coney Island - Bay Ridge,1,NaN
26,27,617,R31,BMT,4th Av,Atlantic Av-Barclays Ctr,Bk,D N R,Subway,40.683666,-73.978810,Manhattan,Coney Island - Bay Ridge,1,NaN
27,28,28,R32,BMT,4th Av,Union St,Bk,R,Subway,40.677316,-73.983110,Manhattan,Bay Ridge - 95 St,0,NaN


In [50]:
#Converting all the station names to lower case
df['station'] = df['station'].str.lower()

In [54]:
#Formating column names for Bronx dataframe
Bronx_df.columns = [column.strip().lower() for column in Bronx_df.columns] #formatting column names
Bronx_df.head()

,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label,ada,ada notes
210,210,210,D01,IND,Concourse,Norwood-205 St,Bx,D,Subway,40.874811,-73.878855,NaN,Manhattan,0,NaN
211,211,211,D03,IND,Concourse,Bedford Park Blvd,Bx,B D,Subway,40.873244,-73.887138,Norwood - 205 St,Manhattan,1,NaN
212,212,212,D04,IND,Concourse,Kingsbridge Rd,Bx,B D,Subway,40.866978,-73.893509,Bedford Pk Blvd & 205 St,Manhattan,1,NaN
213,213,213,D05,IND,Concourse,Fordham Rd,Bx,B D,Subway,40.861296,-73.897749,Bedford Pk Blvd & 205 St,Manhattan,0,NaN
214,214,214,D06,IND,Concourse,182-183 Sts,Bx,B D,Subway,40.856093,-73.900741,Bedford Pk Blvd & 205 St,Manhattan,0,NaN


In [55]:
#Converting all the stop  names to lower case
Bronx_df['stop name'] = Bronx_df['stop name'].str.lower()
Bronx_df.head()


/var/folders/p5/5cz9q1c15pn6hbl2m1csrmjh0000gn/T/ipykernel_70997/2578302608.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bronx_df['stop name'] = Bronx_df['stop name'].str.lower()


,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label,ada,ada notes
210,210,210,D01,IND,Concourse,norwood-205 st,Bx,D,Subway,40.874811,-73.878855,NaN,Manhattan,0,NaN
211,211,211,D03,IND,Concourse,bedford park blvd,Bx,B D,Subway,40.873244,-73.887138,Norwood - 205 St,Manhattan,1,NaN
212,212,212,D04,IND,Concourse,kingsbridge rd,Bx,B D,Subway,40.866978,-73.893509,Bedford Pk Blvd & 205 St,Manhattan,1,NaN
213,213,213,D05,IND,Concourse,fordham rd,Bx,B D,Subway,40.861296,-73.897749,Bedford Pk Blvd & 205 St,Manhattan,0,NaN
214,214,214,D06,IND,Concourse,182-183 sts,Bx,B D,Subway,40.856093,-73.900741,Bedford Pk Blvd & 205 St,Manhattan,0,NaN


In [56]:
#Formating column names for Brooklyn dataframe
Brooklyn_df.columns = [column.strip().lower() for column in Brooklyn_df.columns] #formatting column names
Brooklyn_df.head()

,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label,ada,ada notes
23,24,620,R28,BMT,Broadway,Court St,Bk,R,Subway,40.694100,-73.991777,Manhattan,Bay Ridge - 95 St,0,NaN
24,25,636,R29,BMT,Broadway,Jay St-MetroTech,Bk,R,Subway,40.692180,-73.985942,Manhattan,Bay Ridge - 95 St,1,NaN
25,26,26,R30,BMT,Broadway - Brighton,DeKalb Av,Bk,B Q R,Subway,40.690635,-73.981824,Manhattan,Coney Island - Bay Ridge,1,NaN
26,27,617,R31,BMT,4th Av,Atlantic Av-Barclays Ctr,Bk,D N R,Subway,40.683666,-73.978810,Manhattan,Coney Island - Bay Ridge,1,NaN
27,28,28,R32,BMT,4th Av,Union St,Bk,R,Subway,40.677316,-73.983110,Manhattan,Bay Ridge - 95 St,0,NaN


In [57]:
#Converting all the stop  names to lower case
Brooklyn_df['stop name'] = Brooklyn_df['stop name'].str.lower()
Brooklyn_df.head()

/var/folders/p5/5cz9q1c15pn6hbl2m1csrmjh0000gn/T/ipykernel_70997/3066379176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brooklyn_df['stop name'] = Brooklyn_df['stop name'].str.lower()


,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label,ada,ada notes
23,24,620,R28,BMT,Broadway,court st,Bk,R,Subway,40.694100,-73.991777,Manhattan,Bay Ridge - 95 St,0,NaN
24,25,636,R29,BMT,Broadway,jay st-metrotech,Bk,R,Subway,40.692180,-73.985942,Manhattan,Bay Ridge - 95 St,1,NaN
25,26,26,R30,BMT,Broadway - Brighton,dekalb av,Bk,B Q R,Subway,40.690635,-73.981824,Manhattan,Coney Island - Bay Ridge,1,NaN
26,27,617,R31,BMT,4th Av,atlantic av-barclays ctr,Bk,D N R,Subway,40.683666,-73.978810,Manhattan,Coney Island - Bay Ridge,1,NaN
27,28,28,R32,BMT,4th Av,union st,Bk,R,Subway,40.677316,-73.983110,Manhattan,Bay Ridge - 95 St,0,NaN


In [58]:
df.station.unique()

array(['rit-roosevelt', 'rit-manhattan', 'tompkinsville', 'st. george',
       'eastchster/dyre', 'baychester av', 'gun hill rd', 'pelham pkwy',
       'morris park', 'flatbush av-b.c', 'newkirk av', 'beverly rd',
       'church av', 'winthrop st', 'sterling st', 'president st',
       'new lots av', 'van siclen av', 'pennsylvania av', 'junius st',
       'rockaway av', 'saratoga av', 'sutter av-rutld', 'crown hts-utica',
       'kingston av', 'nostrand av', 'franklin av', 'eastn pkwy-musm',
       'grand army plaz', 'bergen st', 'atl av-barclay', 'nevins st',
       'hoyt st', 'borough hall', 'clark st', '34 st-hudson yd',
       'flushing-main', 'mets-willets pt', '111 st', '103 st-corona',
       'junction blvd', '90 st-elmhurst', '82 st-jackson h',
       '74 st-broadway', '69 st', '61 st woodside', '52 st',
       '46 st bliss st', '40 st lowery st', '33 st-rawson st',
       'astoria ditmars', 'astoria blvd', '30 av', 'broadway', '36 av',
       '39 av', 'queensboro plz', 'court 

In [59]:
Bronx_df["stop name"].unique()

array(['norwood-205 st', 'bedford park blvd', 'kingsbridge rd',
       'fordham rd', '182-183 sts', 'tremont av', '174-175 sts', '170 st',
       '167 st', '161 st-yankee stadium', 'van cortlandt park-242 st',
       '238 st', '231 st', 'pelham bay park', 'buhre av', 'middletown rd',
       'westchester sq-e tremont av', 'zerega av', 'castle hill av',
       'parkchester', 'st lawrence av', 'morrison av-soundview',
       'elder av', 'whitlock av', 'hunts point av', 'longwood av',
       'e 149 st', "e 143 st-st mary's st", 'cypress av', 'brook av',
       '3 av-138 st', 'woodlawn', 'mosholu pkwy',
       'bedford park blvd-lehman college', '183 st', 'burnside av',
       '176 st', 'mt eden av', '149 st-grand concourse',
       '138 st-grand concourse', 'wakefield-241 st', 'nereid av',
       '233 st', '225 st', '219 st', 'gun hill rd', 'burke av',
       'allerton av', 'pelham pkwy', 'bronx park east', 'e 180 st',
       'west farms sq-e tremont av', '174 st', 'freeman st', 'simpson s

In [71]:
df[df["station"]== "morris park"] 

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time
207826,R726,R329,00-00-02,morris park,5,IRT,12/24/2021,20:00:00,REGULAR,6559080,1590163,2021-12-24 20:00:00
207825,R726,R329,00-00-02,morris park,5,IRT,12/24/2021,16:00:00,REGULAR,6559055,1590126,2021-12-24 16:00:00
207824,R726,R329,00-00-02,morris park,5,IRT,12/24/2021,12:00:00,REGULAR,6559018,1590101,2021-12-24 12:00:00
207823,R726,R329,00-00-02,morris park,5,IRT,12/24/2021,08:00:00,REGULAR,6558961,1590088,2021-12-24 08:00:00
207822,R726,R329,00-00-02,morris park,5,IRT,12/24/2021,04:00:00,REGULAR,6558907,1590080,2021-12-24 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2729936,R726,R329,00-00-00,morris park,5,IRT,09/25/2021,17:00:00,REGULAR,1180861,4294989,2021-09-25 17:00:00
2729935,R726,R329,00-00-00,morris park,5,IRT,09/25/2021,13:00:00,REGULAR,1180826,4294840,2021-09-25 13:00:00
2729934,R726,R329,00-00-00,morris park,5,IRT,09/25/2021,09:00:00,REGULAR,1180780,4294767,2021-09-25 09:00:00
2729933,R726,R329,00-00-00,morris park,5,IRT,09/25/2021,05:00:00,REGULAR,1180752,4294732,2021-09-25 05:00:00


In [73]:
# Rename the stop name column to station
Bronx_df = Bronx_df.rename(columns={'stop name': 'station'})
Bronx_df.head(1)

,station id,complex id,gtfs stop id,division,line,station,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label,ada,ada notes
210,210,210,D01,IND,Concourse,norwood-205 st,Bx,D,Subway,40.874811,-73.878855,NaN,Manhattan,0,NaN


In [78]:
#Merge Mta and Bronx_df
merge_mta_Bronx = pd.merge(df,Bronx_df, on=['station'], how='inner')
merge_mta_Bronx.columns


Index(['c_a', 'unit', 'scp', 'station', 'linename', 'division_x', 'date',
       'time', 'desc', 'entries', 'exits', 'date_time', 'station id',
       'complex id', 'gtfs stop id', 'division_y', 'line', 'borough',
       'daytime routes', 'structure', 'gtfs latitude', 'gtfs longitude',
       'north direction label', 'south direction label', 'ada', 'ada notes'],
      dtype='object')

In [79]:
merge_mta_Bronx.drop(['station id', 'complex id','gtfs stop id','division_y',
                      'line','structure', 'north direction label',
                     'south direction label', 'ada', 'ada notes'], axis = 1)

,c_a,unit,scp,station,linename,division_x,date,time,desc,entries,exits,date_time,borough,daytime routes,gtfs latitude,gtfs longitude
0,R729,R292,00-00-02,baychester av,5,IRT,12/24/2021,20:00:00,REGULAR,2064142564,167940229,2021-12-24 20:00:00,Bx,5,40.878663,-73.838591
1,R729,R292,00-00-02,baychester av,5,IRT,12/24/2021,16:00:00,REGULAR,2064142516,167940205,2021-12-24 16:00:00,Bx,5,40.878663,-73.838591
2,R729,R292,00-00-02,baychester av,5,IRT,12/24/2021,12:00:00,REGULAR,2064142443,167940180,2021-12-24 12:00:00,Bx,5,40.878663,-73.838591
3,R729,R292,00-00-02,baychester av,5,IRT,12/24/2021,08:00:00,REGULAR,2064142374,167940151,2021-12-24 08:00:00,Bx,5,40.878663,-73.838591
4,R729,R292,00-00-02,baychester av,5,IRT,12/24/2021,04:00:00,REGULAR,2064142324,167940127,2021-12-24 04:00:00,Bx,5,40.878663,-73.838591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239794,N210,R253,00-00-00,174-175 sts,BD,IND,09/25/2021,16:00:00,REGULAR,767210,307306,2021-09-25 16:00:00,Bx,B D,40.845900,-73.910136
239795,N210,R253,00-00-00,174-175 sts,BD,IND,09/25/2021,12:00:00,REGULAR,767084,307259,2021-09-25 12:00:00,Bx,B D,40.845900,-73.910136
239796,N210,R253,00-00-00,174-175 sts,BD,IND,09/25/2021,08:00:00,REGULAR,766937,307226,2021-09-25 08:00:00,Bx,B D,40.845900,-73.910136
239797,N210,R253,00-00-00,174-175 sts,BD,IND,09/25/2021,04:00:00,REGULAR,766855,307208,2021-09-25 04:00:00,Bx,B D,40.845900,-73.910136
